# Single Cell Decompression Training

## Import Libraries

First, we import the neccessary libraries.

In [1]:
# Import libraries
import anndata as ad
from pathlib import Path
import errno
import os
# import numpy as np
# import pandas as pd

## Helper Functions

Next, we import some helper functions including the cisi segementation training function.

In [5]:
# Helper fncs
import analysis_utils


## CISI
# Import system libraries to configure code directory as module
from os.path import dirname, abspath, join
import sys

# Find code directory relative to our directory
THIS_DIR = dirname('__file__')
CODE_DIR = abspath(join(THIS_DIR, '..', 'code'))
# Add code directory to systems paths
sys.path.append(CODE_DIR)

# Import CISI training fnc.
from train_dictionary_and_compositions import train_U_and_A

## Inputs

In the first part we specify the paths to the input files (.h5ad files created from R SpatialExperiment, TIFF files generated by the steinbock pipeline and panel metadata file) and where the outputs should be stored.

In [6]:
# Specify input paths
training_data_path = Path('/mnt/bb_dqbm_volume')
spe_path = Path(os.path.join(training_data_path, 
                             'data/Tonsil_th152/preprocessed_data/spe.h5ad'))
tiffs_path = Path(os.path.join(training_data_path, 'data/Tonsil_th152/steinbock/img'))
panel_path = Path(os.path.join(training_data_path, 'data/Tonsil_th152/steinbock/panel.csv'))

# Specify output path
out_path = Path(os.path.join(training_data_path, 'analysis/Tonsil_th152/example'))

# Create output directory if it doesn't exist
out_path.mkdir(parents=True, exist_ok=True)

In [7]:
# Check that input files/dictionary exist
if not analysis_utils.is_valid_file(spe_path, ['.h5ad']):
    # If file is not found, throw error
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT),
                            spe_path)
if not analysis_utils.is_valid_file(panel_path, ['.csv']):
    # If file is not found, throw error
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT),
                            panel_path)
if not analysis_utils.is_valid_directory(tiffs_path):
    # If directory is not found or doesn't contain TIFF files, throw an error
    raise Exception('Input TIFFS path {0} directory does not exist or does\
    not contain any valid TIFF files.'.format(tiffs_path)) from FileNotFoundError

Next, we read in the input files for training. For this we have the Tonsil th152 dataset consisting of 5 ROIs and we read the data in once it has been processed by steinbock into segmented single cells and once using the hot pixel corrected TIFF image files directly. From this we create one numpy array of channels x cells and one numpy array containing channels vs pixels.

In [8]:
# Read in SpatialExperiment converted to anndata by cellconverter in R
spe = ad.read_h5ad(spe_path)
# Remove uninteresting proteins/channels
spe = spe[:, ~spe.var.index.str.contains('Histone|Ir[0-9]', regex=True, case=False)]
print(spe)

AnnData object with n_obs × n_vars = 162624 × 43
    obs: 'sample_id', 'ObjectNumber', 'area', 'major_axis_length', 'minor_axis_length', 'eccentricity', 'width_px', 'height_px', 'ROI'
    var: 'channel', 'name', 'keep', 'ilastik', 'deepcell', 'Tube.Number', 'Metal', 'use_channel'
    uns: 'X_name'
    layers: 'exprs', 'log_exprs'


In [10]:
images = analysis_utils.anndata_from_tiff(tiffs_path, panel_path)
# Remove uninteresting proteins/channels
images = images[:, ~images.var.index.str.contains('Histone|Ir[0-9]', regex=True, case=False)]
print(images)


AnnData object with n_obs × n_vars = 9250000 × 43
    obs: 'sample_id', 'ObjectNumber', 'ROI'
    var: 'channel', 'name', 'keep', 'ilastik', 'deepcell', 'Tube Number', 'Metal'


In [11]:
training_res, training_res_comp, cor, cond_prob = train_U_and_A(spe, outpath,
                                                                test_set=('20220520_TsH_th152_cisi1_001',))

on 2: 10 measurements                                                                                                   
on 2: 0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.                                        
       0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
       0. 0.]
on 2: 100 [0.16974001 0.18408223 0.21928707 0.22142798 0.16865808 0.18413018                                                       | ▂▄▆ 2/4 [50%] in 1:35 (0.0/s, eta: 10s) ▄▂▂ 2/4 [50%] in 2:20 (0.0/s, eta: 10s) 
       0.21082242 0.18299919 0.24525927 0.19746585 0.20086209 0.21878983
       0.22484721 0.16608197 0.1892551  0.189547   0.16233854 0.17607533
       0.21071741 0.2153271  0.18732127 0.16899833 0.16367787 0.17052181
       0.20973286 0.19817968 0.17188679 0.19962898 0.20477844 0.16203738
       0.19299765 0.17780573 0.1772823  0.18316254 0.22652478 0.22010189
       0.2502961  0.20353152 0.16847974 0.24847728 0.1756802  0.18096602
       0.1786

on 2: 1200 [0.23362891 0.24163992 0.27546177 0.27392364 0.23431336 0.23455472                                           a: 10s)  2/4 [50%] in 26:18 (0.0/s, eta: 10s) 
       0.23978485 0.24894849 0.24525927 0.25811183 0.24090412 0.23942409
       0.23826665 0.2445304  0.24400442 0.24003167 0.23712476 0.24697534
       0.26674387 0.23999726 0.2554887  0.26113047 0.23543348 0.24087695
       0.24736789 0.2350834  0.23940331 0.2402011  0.24930543 0.24821141
       0.23484393 0.26631842 0.23971731 0.26732103 0.23762775 0.24213809
       0.2502961  0.24470112 0.24808702 0.24847728 0.24985444 0.23515167
       0.25045863 0.23274398 0.24697752 0.23448797 0.24195283 0.2502103
       0.23310116 0.23242897]
on 2: 1300 [0.23362891 0.24163992 0.27546177 0.27392364 0.23431336 0.23455472                                           
       0.23978485 0.24894849 0.24525927 0.25811183 0.24090412 0.23942409
       0.23826665 0.2445304  0.24400442 0.24003167 0.23712476 0.24697534
       0.26674387 0.239997

on 2: /home/ubuntu/anaconda3/envs/cisi_imc_env/lib/python3.10/site-packages/scipy/spatial/distance.py:630: RuntimeWarning: invalid value encountered in double_scalars
        dist = 1.0 - uv / np.sqrt(uu * vv)


Fold 0 |████████████████████████████████████████| 4/4 [100%] in 43:55.0 (0.00/s)                                        
on 2: 10 measurements                                                                                                   
on 2: 0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.                                        
       0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
       0. 0.]
on 2: 100 [0.22174271 0.21121633 0.25137487 0.18922006 0.21697675 0.21836679                                             eta: 10s)  █▆▄ 2/4 [50%] in 2:13 (0.0/s, eta: 10s) 
       0.21124384 0.19094113 0.21140662 0.23230636 0.23994203 0.18877719
       0.20549501 0.26915272 0.18336276 0.22582283 0.19574838 0.2599391
       0.21718431 0.20429684 0.20676311 0.19380158 0.20453635 0.22053711
       0.20960635 0.18266013 0.26343895 0.2456773  0.20743964 0.23191105
       0.22972254 0.22214709 0.19277477 0.22596477 0.18646692 0.21600128
       

on 2: 1300 [0.27870072 0.25790647 0.2614614  0.26766438 0.29437384 0.27128705                                           
       0.27502912 0.26652679 0.26254952 0.27754459 0.26947598 0.26115205
       0.26215733 0.26915272 0.25841329 0.26275239 0.26117238 0.2599391
       0.26078288 0.26252514 0.2935065  0.27010325 0.26277227 0.26029577
       0.2588252  0.26424095 0.26343895 0.25850008 0.26437983 0.2740425
       0.2773919  0.27050743 0.25898774 0.26943001 0.26000714 0.26113562
       0.26425833 0.26195295 0.26211319 0.29072811 0.28847274 0.26512223
       0.26451749 0.26698365 0.3316375  0.26023295 0.26362905 0.30927895
       0.26587637 0.27123966]
on 2: 1400 [0.27870072 0.26475114 0.2614614  0.26766438 0.29437384 0.27128705                                           
       0.27502912 0.26652679 0.26254952 0.27754459 0.26947598 0.26115205
       0.26215733 0.26915272 0.27199988 0.26275239 0.26117238 0.2599391
       0.26078288 0.26252514 0.2935065  0.27010325 0.26277227 0.26029577
 

...

Fold 2 |████████████████████████████████████████| 4/4 [100%] in 37:03.8 (0.00/s)                                        
on 2: 10 measurements                                                                                                   
on 2: 0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.                                        
       0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
       0. 0.]
on 2: 100 [0.26548599 0.25850295 0.2749356  0.25965276 0.21680378 0.22421338                                             7s) |████████████████████                    | ▆█▆ 2/4 [50%] in 2:49 (0.0/s, eta: 7s) |████████████████████                    | ▄▂▂ 2/4 [50%] in 4:20 (0.0/s, eta: 7s) 2/4 [50%] in 4:58 (0.0/s, eta: 7s) 
       0.24885066 0.22503675 0.22743647 0.23673497 0.28518785 0.24127334
       0.220653   0.26851427 0.22346055 0.231354   0.24334312 0.24192855
       0.24394588 0.28400963 0.30150482 0.2531409  0.29815853 0.24573195


on 2: 1600 [0.294355   0.30255965 0.31101337 0.30562664 0.30503843 0.32123782                                           .0/s, eta: 7s) 2/4 [50%] in 1:13:30 (0.0/s, eta: 7s) 2/4 [50%] in 1:14:42 (0.0/s, eta: 7s) |████████████████████                    | ▃▅▇ 2/4 [50%] in 1:15:17 (0.0/s, eta: 7s) |████████████████████                    | ▆▄▂ 2/4 [50%] in 1:15:26 (0.0/s, eta: 7s) |████████████████████                    | ▄▂▂ 2/4 [50%] in 1:15:29 (0.0/s, eta: 7s) |████████████████████                    | ▂▄▆ 2/4 [50%] in 1:16:41 (0.0/s, eta: 7s) |████████████████████                    | ▄▆█ 2/4 [50%] in 1:16:59 (0.0/s, eta: 7s) 
       0.30095638 0.29978354 0.29590962 0.29515145 0.30171867 0.30957022
       0.31388355 0.29127133 0.34200699 0.3056374  0.29257413 0.29866745
       0.29171743 0.30348384 0.30150482 0.31235721 0.29815853 0.31240282
       0.30747597 0.29698582 0.31282465 0.29208019 0.32005683 0.2952078
       0.30784284 0.29378247 0.31082785 0.3013832  0.29926903 0.30191134

on 2: /home/ubuntu/anaconda3/envs/cisi_imc_env/lib/python3.10/site-packages/scipy/spatial/distance.py:630: RuntimeWarning: invalid value encountered in double_scalars
        dist = 1.0 - uv / np.sqrt(uu * vv)


Fold 3 |████████████████████████████████████████| 4/4 [100%] in 1:36:55.1 (0.00/s)                                      
Cross-fold validations done.
The pearson correlations are: [0.2754617713258924, 0.3316374956032279, 0.34363825868768666, 0.3420069885670424]


ValueError: A 2-dimensional array must be passed.